In [1]:
import tensorflow as tf
import dlib
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (96,96))
    # Scale image to be between 0 and 1 
    img = img / 255.0
    img = tf.expand_dims(img, 0)
    # Return image
    return img

In [3]:
# Using TFLIte without OPTIMIZE
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

In [4]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [5]:
input_details

[{'name': 'serving_default_conv2d_8_input:0',
  'index': 0,
  'shape': array([ 1, 96, 96,  3], dtype=int32),
  'shape_signature': array([-1, 96, 96,  3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [6]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 28,
  'shape': array([1, 4], dtype=int32),
  'shape_signature': array([-1,  4], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [39]:
# input_img = preprocess(os.path.join("dataset", "cropped", "arham", "9a1b3c24-99df-11ec-ab0e-842b2bb3a5b6.jpg"))
# input_img = preprocess(os.path.join("dataset", "cropped", "mohsin", "0a2bbaa0-99e0-11ec-826f-842b2bb3a5b6.jpg"))
input_img = preprocess(os.path.join("dataset", "cropped", "ehsan", "4a02d925-99e0-11ec-be8c-842b2bb3a5b6.jpg"))

In [18]:
def predict(input_img):
    input_img = input_img.astype("float32")
    input_img = tf.expand_dims(input_img, 0)
    interpreter.set_tensor(input_details[0]['index'], input_img)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_details[0]['index'])
    classnames = ['arham', 'ehsan', 'mohsin', 'unknown']    
    return classnames[np.argmax(pred)]

In [8]:
# Extracting Bounding Boxes
def convert_and_trim_bb(image, rect):
    # extract the starting and ending (x, y)-coordinates of the
    # bounding box
    startX = rect.left()
    startY = rect.top()
    endX = rect.right()
    endY = rect.bottom()
    # ensure the bounding box coordinates fall within the spatial
    # dimensions of the image
    startX = max(0, startX)
    startY = max(0, startY)
    endX = min(endX, image.shape[1])
    endY = min(endY, image.shape[0])
    # compute the width and height of the bounding box
    w = endX - startX
    h = endY - startY
    # return our bounding box coordinates
    return (startX, startY, w, h)

In [9]:
detector = dlib.get_frontal_face_detector()

In [10]:
def gen_faces(image):
    """
    Function that gets image as input and generate face from it and returns
    face from the image.
    """
    image_height, image_width, _ = image.shape
    faces = []
    detections = detector(image)
    for i, d in enumerate(detections):
        # Croping Face from image
        crop_img = image[max(0, d.top()): min(d.bottom(), image_height),
                    max(0, d.left()): min(d.right(), image_width)]
        faces.append(crop_img)
#     returning array of faces
    return faces

In [11]:
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (50, 50)
# fontScale
fontScale = 1
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2

In [19]:
stream = cv2.VideoCapture("dataset/negative.mp4")

while True:
    ret, frame = stream.read()
    if not ret:
        break
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    im = cv2.resize(rgb, None, fx=1/2, fy=1/2, interpolation=cv2.INTER_AREA)
    rects = detector(im)
    boxes = [convert_and_trim_bb(rgb, r) for r in rects]
    face = gen_faces(im)[0]
    face = cv2.resize(face, (96, 96))
    face = face / 255.
    prediction = predict(face)
    for (x, y, w, h) in boxes:
        cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    im = cv2.putText(im, prediction.capitalize(), org, font, fontScale, color, thickness, cv2.LINE_AA)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) 
    cv2.imshow("Feed", im)    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

stream.release()
cv2.destroyAllWindows()